In [ ]:
#default_exp fastai.transform

In [ ]:
#hide
from nbdev.showdoc import *

# Transform Errors
> General errors for dealing with `Transform`s

In [ ]:
#export
from fastai.data.core import TfmdDL

from fastcore.basics import patch
from fastcore.meta import delegates
from fastcore.transform import _get_name, Transform

Transform errors are currently some of the hardest to debug, including the dreaded `Could not do one pass through your DataLoader`.

This module provides some extra errors and injection points to see the full traces and their causes

In [ ]:
#export
def transform_error(e:Exception, nm:str, event:str) -> Exception:
    """
    Raises Exception `e` stemming from a Transform with more information
    
    - `nm`: The name of the Transform
    - `event`: The event called (such as `encodes` or `decodes`)
    """
    err = f'There was an issue calling event {event} on transform {nm}:\n\n'
    err += e.args[0]
    e.args = [err]
    raise e

In [ ]:
#export
@patch
def __call__(self:Transform, x, **kwargs):
    try:
        return self._call('encodes', x, **kwargs)
    except Exception as e:
        transform_error(e, _get_name(self), 'encodes')

In [ ]:
#export
@patch
def decode(self:Transform, x, **kwargs):
    "Delegate to <code>decodes</code> to undo transform"
    try:
        return self._call('decodes', x, **kwargs)
    except Exception as e:
        transform_error(e, _get_name(self), 'decodes')

The `transform_error` is extremely useful for trying to figure out what section of your pipeline broke.

As an example, we'll write a broken transform and attempt to build a `DataLoader` with it.

> Note: We have added in the capability to get a more verbose error message when building the `DataLoaders` inplace with this module. Check the source notebook or code to see the `@patch`'d code

In [ ]:
#export
@patch
@delegates(DataLoader.new)
def new(self:TfmdDL, dataset=None, cls=None, **kwargs):
    "Create a new version of self with a few changed attributes"
    res = super(TfmdDL, self).new(dataset, cls, do_setup=False, **kwargs)
    if not hasattr(self, '_n_inp') or not hasattr(self, '_types'):
        try:
            self._one_pass()
            res._n_inp,res._types = self._n_inp,self._types
        except Exception as e: 
            print("Could not do one pass in your dataloader, there is something wrong in it")
            raise e
    else: res._n_inp,res._types = self._n_inp,self._types
    return res

In [ ]:
from fastai.vision.all import *

class BrokenTransform(DisplayedTransform):
    "A purposefully broken transform"
    y = 'a'
    def encodes(self, x:TensorImage): return x*self.y

In [ ]:
#failing
path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(),
    item_tfms=[Resize(224)], batch_tfms=[BrokenTransform()])

x,y = dls.one_batch()

And as you can see it tells that there was an issue on our `encodes` for `BrokenTransform`, whichi s exactly what we expected to have happen!